<a href="https://colab.research.google.com/github/qamtam/Hands-on-machine-learning/blob/main/CH13_SET11_with_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tfx==0.21.2
    print("You can safely ignore the package incompatibility errors.")
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty pony figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "data"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

BytesList = tf.train.BytesList
FloatList = tf.train.FloatList
Int64List = tf.train.Int64List
Feature = tf.train.Feature
Features = tf.train.Features
Example = tf.train.Example


You can safely ignore the package incompatibility errors.


In [ ]:
"""
Exercise: Download the Large Movie Review Dataset, which contains 50,000 movies reviews from the Internet Movie Database. The data is organized in two directories, train and test, each containing a pos subdirectory with 12,500 positive reviews and a neg subdirectory with 12,500 negative reviews. Each review is stored in a separate text file. There are other files and folders (including preprocessed bag-of-words), but we will ignore them in this exercise."""


#to-do
#download dataset
#split test into validation and test 15000/10000
#use tf.data to create an efficiently loading dataset (batch preload)


from pathlib import Path

DOWNLOAD_ROOT = "http://ai.stanford.edu/~amaas/data/sentiment/"
FILENAME = "aclImdb_v1.tar.gz"
filepath = keras.utils.get_file(FILENAME, DOWNLOAD_ROOT + FILENAME, extract=True)
path = Path(filepath).parent / "aclImdb"
path


PosixPath('/root/.keras/datasets/aclImdb')

In [ ]:
import os
i=0
for name, subdirs, files in os.walk(path):
  indent = len(Path(name).parts) - len(path.parts)
  if i == 0:
    print(name)
    print(Path(name))
    print(Path(name).parts)
    print(Path.parts)
  i +=1
  print("    " * indent + Path(name).parts[-1] + os.sep)
  for index, filename in enumerate(sorted(files)):
    if index ==3:
            print("    " * (indent + 1) + "...")
            break
    print("    " * (indent + 1) + filename)   


/root/.keras/datasets/aclImdb
/root/.keras/datasets/aclImdb
('/', 'root', '.keras', 'datasets', 'aclImdb')
aclImdb/
    README
    imdb.vocab
    imdbEr.txt
    test/
        labeledBow.feat
        urls_neg.txt
        urls_pos.txt
        pos/
            0_10.txt
            10000_7.txt
            10001_9.txt
            ...
        neg/
            0_2.txt
            10000_4.txt
            10001_1.txt
            ...
    train/
        labeledBow.feat
        unsupBow.feat
        urls_neg.txt
        ...
        unsup/
            0_0.txt
            10000_0.txt
            10001_0.txt
            ...
        pos/
            0_9.txt
            10000_8.txt
            10001_10.txt
            ...
        neg/
            0_3.txt
            10000_4.txt
            10001_4.txt
            ...


In [ ]:
def review_paths(dirpath):
    print(dirpath)
    return [str(path) for path in dirpath.glob("*.txt")] # path.glob zwraca wszystkie pliki i foldery z danym wzorcem

In [ ]:

train_pos = review_paths(path / "train" / "pos")
train_neg = review_paths(path / "train" / "neg")
test_valid_pos = review_paths(path / "test" / "pos")
test_valid_neg = review_paths(path / "test" / "neg")
train_pos[1]

/root/.keras/datasets/aclImdb/train/pos
/root/.keras/datasets/aclImdb/train/neg
/root/.keras/datasets/aclImdb/test/pos
/root/.keras/datasets/aclImdb/test/neg


'/root/.keras/datasets/aclImdb/train/pos/6720_9.txt'

In [ ]:
print(print(glob.glob("/root/.keras/datasets/aclImdb/train")))

NameError: ignored

In [ ]:
import glob
path_pos = path / "train" / "pos"

print(list((path / "train" / "pos").glob('*')))

In [ ]:
path


In [ ]:
[x for x in path.parent.iterdir() if x.exists()]

In [ ]:
path

In [ ]:
np.random.shuffle(test_valid_pos)
np.random.shuffle(test_valid_neg)
len(test_valid_pos)
# validation 7500 
# test pierwsze 5000
test_pos = test_valid_pos[:5000]
test_neg = test_valid_neg[:5000]
valid_pos = test_valid_pos[5000:]
valid_neg = test_valid_neg[5000:]

In [ ]:
test_pos

In [ ]:
all_paths

In [ ]:
def make_dataset(filepaths):
  #dataset_paths = tf.data.Dataset.list_files(filepaths[:100])
  dataset = tf.data.TextLineDataset(filepaths)

  dataset = dataset.shuffle(5000).repeat(1)
  dataset = dataset.batch(32).prefetch(1)
  return dataset
#forgot about labels, oof

In [ ]:
X_train = make_dataset(train_pos)

In [ ]:
for x in X_train.take(1):
  print(x)

tf.Tensor(
[b"My personal favorite horror film. From the lengthy first tracking shot to the final story twist, this is Carpenter's masterpiece.<br /><br />Halloween night 1963, little Michael Meyers murders his older sister. All-hallows-eve 1978, Michael escapes from Smith's Grove sanitarium. Halloween night, Michael has come home to murder again.<br /><br />The story is perfectly simple, Michael stalks and kills babysitters. No bells or whistles, just the basics. It's Carpenter's almost over-powering atmosphere of dread that generates the tension. Like any great horror film, events are telegraphed long in advance, yet they still seem to occur at random, never allowing the audience to the chance to second guess the film.<br /><br />The dark lighting, the long steady-cam shots, and (most importantly) that damn eerie music create the most claustrophobic and uncomfortable scenes I have yet to see in film. There is a body count, but compared to the slew of slashers after this it's fairly s

In [ ]:
#once again 
#full python solution, we know it fits in memory
def imdb_python_dataset(filepaths_positive, filepaths_negative):
  reviews = []
  labels = []
  for filepaths, label in ((filepaths_negative, 0), filepaths_positive, 1):
    for filepath in filepaths:
      with open(filepath) as review_file:
        reviews.append(review_file.read())
      labels.append(label)
  reviews_dataset = tf.constant(reviews) # here we make actual tf compatible data
  labels_dataset = tf.constant(labels)
  full_dataset = tf.data.Dataset.from_tensor_slices((reviews_dataset, labels_dataset)) #double parantheses because of set of tensors
  return full_dataset

#tf.data
def imdb_dataset(filepaths_positive, filepaths_negative, n_read_threads=5):
  dataset_neg = tf.data.TextLineDataset(filepaths_negative, num_parallel_reads=n_read_threads) #text line dataset takes as an argument dataPATHS
  dataset_neg = dataset_neg.map(lambda review: (review, 0)) #attach label
  dataset_pos = tf.data.TextLineDataset(filepaths_positive, num_parallel_reads=n_read_threads)
  dataset_pos = dataset_pos.map(lambda review: (review, 1))
  return tf.data.Dataset.concatenate(dataset_pos, dataset_neg)



In [ ]:
%timeit -r1 for X, y in imdb_dataset(train_pos, train_neg).repeat(10): pass #co ma zrobić w pętli for? pass, czyli nic - a poza tym nie trzeba zaczynać nowej linjki

1 loop, best of 1: 1min 20s per loop


In [ ]:
%timeit -r1 for X, y in imdb_dataset(train_pos, train_neg).cache().repeat(1): pass

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)

In [ ]:
%timeit -n1 -r1 print("yo mama")

SyntaxError: ignored

In [ ]:
batch_size = 32

train_set = imdb_dataset(train_pos, train_neg).shuffle(25000).batch(batch_size).prefetch(1)
valid_set = imdb_dataset(valid_pos, valid_neg).batch(batch_size).prefetch(1)
test_set = imdb_dataset(test_pos, test_neg).batch(batch_size).prefetch(1)
single_tensor_test_set = imdb_dataset(test_pos, test_neg)

In [ ]:
max_features = 100000
max_len = 1
embedding_dims = 3
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vectorize_layer = TextVectorization(
    max_tokens = max_features,
    output_mode = 'int',
    output_sequence_length = max_len
)

ValueError: ignored

<MapDataset shapes: (), types: tf.string>

ValueError: ignored

In [ ]:

vectorize_layer.adapt(single_tensor_test_set)

In [ ]:
for x in single_tensor_test_set.take(3):
  print(x)

tf.Tensor(b'It infuriates me no end that, now and forever, I will have to identify this movie (which I consider a masterpiece, and I don\'t use that word lightly) with the qualifier "Not the Michael Douglas movie!" Not only are the titles the same, but they refer to the same thing- the radioactive fallout that rained upon the survivors of the first nuclear bombings. In Imamura\'s film, this is no cheap metaphor; the whole movie is about the fallout, physical and emotional, from Hiroshima and the war itself. As the deterioration of a couple and their grown niece becomes more grimly clear, the ironic imagery becomes more potent, from the old clock that is reset each night to the stone gods that gradually pile up outside the heroine\'s door. (These, in turn, are carved by a shellshocked veteran who is compelled, in a series of tragicomic episodes, to attack anything with a motor that approaches the town.) The bombing day itself is shown in piecemeal flashbacks that are coolly horrifying. 

In [ ]:
vectorize_layer(["you"])

<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[23]])>

In [ ]:
vectorize_layer(X_train)

ValueError: ignored

In [ ]:
sample_review_batches = train_set.map(lambda review, label: review) # text vectorization się uczy na "pojedynczych tensorach"
sample_reviews = np.concatenate(list(sample_review_batches.as_numpy_iterator()),
                                axis=0)
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D
text_vectorization = TextVectorization(
    max_tokens = max_features,
    output_mode = 'int',
    output_sequence_length = 100
)
text_vectorization.adapt(sample_reviews)
model = keras.models.Sequential([
                                 text_vectorization, #po przystosowaniu na gołych recenzjach może być akceptowana w modelu
                                 Embedding(100000, 16), # wymagane, żeby zaczął się uczyć - model nie lubi kodowania zerojedynkowego
                                 GlobalAveragePooling1D(), #wyciąga średnią ze zdań - w wierszach są zakodowane słowa,  a liczy średnią zawsze po kolumnach
                                 # zdania mają różną długość, to ten problem redukuje
                                 # też wymagane by się zaczął uczyć
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(1, activation="sigmoid")
])

model.compile(metrics=["accuracy"], loss="binary_crossentropy", optimizer="nadam")
model.fit(train_set, epochs=25, validation_data=valid_set)

Epoch 1/25
782/782 [==============================] - 16s 21ms/step - loss: 0.4785 - accuracy: 0.7722 - val_loss: 0.3983 - val_accuracy: 0.8201
Epoch 2/25
782/782 [==============================] - 16s 20ms/step - loss: 0.2770 - accuracy: 0.8886 - val_loss: 0.4077 - val_accuracy: 0.8177
Epoch 3/25
782/782 [==============================] - 16s 20ms/step - loss: 0.1831 - accuracy: 0.9322 - val_loss: 0.4596 - val_accuracy: 0.8127
Epoch 4/25
782/782 [==============================] - 16s 21ms/step - loss: 0.1164 - accuracy: 0.9608 - val_loss: 0.5510 - val_accuracy: 0.8001
Epoch 5/25
782/782 [==============================] - 16s 20ms/step - loss: 0.0705 - accuracy: 0.9789 - val_loss: 0.6465 - val_accuracy: 0.7977
Epoch 6/25
782/782 [==============================] - 16s 20ms/step - loss: 0.0437 - accuracy: 0.9882 - val_loss: 0.7730 - val_accuracy: 0.7869
Epoch 7/25
782/782 [==============================] - 16s 20ms/step - loss: 0.0258 - accuracy: 0.9938 - val_loss: 0.8981 - val_accuracy:

In [ ]:
input_shape = (2, 2, 1)
x = tf.random.normal(input_shape)
print(x)
y = tf.keras.layers.GlobalAveragePooling1D()(x)
print(y.shape)
print(y)

tf.Tensor(
[[[ 0.3798573 ]
  [-0.97245353]]

 [[ 0.88101804]
  [ 1.2399622 ]]], shape=(2, 2, 1), dtype=float32)
(2, 1)
tf.Tensor(
[[-0.29629812]
 [ 1.0604901 ]], shape=(2, 1), dtype=float32)


In [ ]:
import tensorflow_datasets as tfds

datasets = tfds.load(name="imdb_reviews")
train_set, test_set = datasets["train"], datasets["test"]

for example in train_set.take(1):
    print(example["text"])
    print(example["label"])

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDC01N7/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDC01N7/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDC01N7/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)
